In [1]:
import pandas as pd
#!pip3 install --upgrade pandas
import datetime

# print non truncated column info in pandas dataframe
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 500)

## -- Categories

In [2]:
# read the categories tree
df_cat_tree = pd.read_csv('data/category_tree.csv')

In [3]:
df_cat_tree.head(3)

,categoryid,parentid
0,1016,213.0
1,809,169.0
2,570,9.0


In [4]:
df_cat_tree.shape

(1669, 2)

In [5]:
#df_cat_tree = df_cat_tree.drop_duplicates(subset='categoryid', keep='first')

In [6]:
df_cat_tree.categoryid.nunique()

1669

In [7]:
df_cat_tree.parentid.nunique()

362

In [8]:
df_cat_tree['categoryid'] = df_cat_tree['categoryid'].astype('int')

## -- Events

In [9]:
# read the events (views, addtocart & buy)
df_events = pd.read_csv('data/events.csv')

### --- Create the sessions

In [10]:
# keep only view events
df_view = df_events[df_events.event=='view']

In [11]:
# create a column with the real timestamp
df_view['timestamp_real'] = df_view['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(x//1000.0))

C:\Users\Dimos\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
# create a column with the yearly number of the day
df_view['day_yearly'] = df_view['timestamp_real'].apply(lambda x: x.isocalendar()[1]*x.isocalendar()[2])

C:\Users\Dimos\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [13]:
# sort by date
df_view = df_view.sort_values(by=['day_yearly', 'timestamp_real'],ascending=True)

In [14]:
df_view.head(2)

,timestamp,visitorid,event,itemid,transactionid,timestamp_real,day_yearly
1476946,1430686807392,167772,view,215503,NaN,2015-05-04 00:00:07,19
1472454,1430686822425,648236,view,239419,NaN,2015-05-04 00:00:22,19


In [23]:
df_view.shape

(2664312, 7)

In [24]:
# group all products that each visitor viewed per day
df_sessions = df_view.groupby(['visitorid', 'day_yearly'],as_index=False)['itemid','timestamp_real'].agg(lambda x: list(x))

In [25]:
# create a column with the number of views for each visitor id per day
df_sessions['N_views'] = df_sessions['itemid'].apply(lambda x: len(x))

In [26]:
df_sessions.describe()

,visitorid,day_yearly,N_views
count,1.646142e+06,1.646142e+06,1.646142e+06
mean,7.041208e+05,1.068037e+02,1.618519e+00
std,4.064136e+05,5.914570e+01,2.927082e+00
min,0.000000e+00,1.900000e+01,1.000000e+00
25%,3.521992e+05,5.800000e+01,1.000000e+00
50%,7.044630e+05,1.000000e+02,1.000000e+00
75%,1.056116e+06,1.480000e+02,1.000000e+00
max,1.407579e+06,2.590000e+02,3.510000e+02


In [27]:
x0 = len(df_sessions[df_sessions.N_views==1])
x_all = len(df_sessions)

print('The', round(100*(x0/x_all),2), '% of the users viewed only 1 product and then left the site.')

The 76.54 % of the users viewed only 1 product and then left the site.


In [28]:
x1 = len(df_sessions[df_sessions.N_views==2])
x_all = len(df_sessions)

print('The', round(100*(x1/x_all),2), '% of the users viewed only 2 products and then left the site.')

The 13.02 % of the users viewed only 2 products and then left the site.


In [29]:
x2 = len(df_sessions[df_sessions.N_views==3])
x_all = len(df_sessions)

print('The', round(100*(x2/x_all),2), '% of the users viewed only 3 products and then left the site.')

The 4.53 % of the users viewed only 3 products and then left the site.


In [30]:
x3 = len(df_sessions[df_sessions.N_views>3])
x_all = len(df_sessions)

print('The', round(100*(x3/x_all),2), '% of the users viewed more than 3 products before leaving the site.')

The 5.91 % of the users viewed more than 3 products before leaving the site.


In [31]:
df_sessions.head(2)

,visitorid,day_yearly,itemid,timestamp_real,N_views
0,0,185,"[285930, 357564, 67045]","[2015-09-11 23:49:49, 2015-09-11 23:52:39, 2015-09-11 23:55:17]",3
1,1,132,[72028],[2015-08-13 20:46:06],1


In [32]:
df_sessions.to_pickle('data/final/df_sessions_new.pkl')

### --- Create the links

In [33]:
df_sessions = pd.read_pickle('data/final/df_sessions_new.pkl')

In [34]:
# drop visitors with only 1 view
df_sessions = df_sessions[df_sessions.N_views>1]

In [35]:
sessions = df_sessions.itemid.to_list()
len(sessions)

386240

In [36]:
# set the window size 
# e.g. window_size=1 means that only the next product is linked with the 'current' product
# e.g. window_size=2 means that the next 2 products are linked with the 'current' product
window_size = 1

In [37]:
df_sessions.head()

,visitorid,day_yearly,itemid,timestamp_real,N_views
0,0,185,"[285930, 357564, 67045]","[2015-09-11 23:49:49, 2015-09-11 23:52:39, 2015-09-11 23:55:17]",3
2,2,160,"[325215, 325215, 259884, 216305, 342816, 342816, 216305, 325215]","[2015-08-07 20:51:44, 2015-08-07 20:53:33, 2015-08-07 20:56:52, 2015-08-07 21:01:08, 2015-08-07 21:08:25, 2015-08-07 21:17:24, 2015-08-07 21:17:43, 2015-08-07 21:20:57]",8
7,6,245,"[253615, 344723, 344723, 344723]","[2015-08-30 09:39:38, 2015-08-30 09:40:23, 2015-08-30 09:54:09, 2015-08-30 09:56:33]",4
8,7,80,"[139394, 164941]","[2015-05-14 08:39:36, 2015-05-14 08:42:42]",2
15,13,100,"[320589, 278689]","[2015-06-18 07:42:22, 2015-06-18 07:43:15]",2


In [38]:
links = []
for session in sessions:
    for i in range(len(session)-1):
        for j in range(window_size):
            try:
                link = [session[i],session[i+1+j]]
                links.append(link)
            except:
                pass
        

In [39]:
# covert list of lists to dataframe
df_links1 = pd.DataFrame(links, columns=['product1', 'product2'])

In [40]:
# covert list of lists to dataframe with the opposite sequence
df_links2 = pd.DataFrame(links, columns=['product2', 'product1'])

In [41]:
# concat the 2 dataframes
df_links = pd.concat([df_links1, df_links2], axis=0)

C:\Users\Dimos\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [42]:
# drop duplicates
df_links = df_links.drop_duplicates(subset=['product1', 'product2'], keep='first')

In [43]:
df_links.head(2)

,product1,product2
0,285930,357564
1,357564,67045


In [44]:
df_links.shape

(998750, 2)

#### ---- Add the category and subcategory of each product

In [45]:
# read the item properties
df_item_properties_part1 = pd.read_csv('data/item_properties_part1.csv')
df_item_properties_part2 = pd.read_csv('data/item_properties_part2.csv')

# join the 2 dfs
df_item_properties = pd.concat([df_item_properties_part1, df_item_properties_part2])

In [46]:
# create a df with the subcategory of each product
df_categs = df_item_properties[df_item_properties.property=='categoryid']

# keep only one row for each pair (itemid, property)
df_categs = df_categs.drop_duplicates(subset=['itemid', 'property'], keep='first')

# convert category_ids to int
df_categs['value'] = df_categs['value'].astype('int')

# add the parentid to each category
df_categs = df_categs.merge(df_cat_tree, left_on='value', right_on='categoryid', how='left')

# keep only specific columns
df_categs = df_categs[['itemid', 'categoryid', 'parentid']]

In [47]:
df_categs = df_categs.dropna(subset=['parentid'])
df_categs['parentid'] = df_categs['parentid'].astype('int')

In [48]:
df_categs.head(2)

,itemid,categoryid,parentid
0,460429,1338.0,1278
1,281245,1277.0,312


In [49]:
# add the category and subcategory of product 1
df_links = df_links.merge(df_categs, left_on='product1', right_on='itemid', how='left')
# drop 'itemid' column
df_links = df_links.drop(['itemid'], axis=1)
# rename columns
df_links.columns = ['product1', 'product2', 'subcategory_p1', 'category_p1']

In [50]:
# add the category and subcategory of product 2
df_links = df_links.merge(df_categs, left_on='product2', right_on='itemid', how='left')
# drop 'itemid' column
df_links = df_links.drop(['itemid'], axis=1)
# rename columns
df_links.columns = ['product1', 'product2', 'subcategory_p1', 'category_p1', 'subcategory_p2', 'category_p2']

In [51]:
df_links.head()

,product1,product2,subcategory_p1,category_p1,subcategory_p2,category_p2
0,285930,357564,1188.0,1497.0,256.0,1257.0
1,357564,67045,256.0,1257.0,333.0,1497.0
2,325215,325215,299.0,73.0,299.0,73.0
3,325215,259884,299.0,73.0,299.0,73.0
4,259884,216305,299.0,73.0,299.0,73.0


In [52]:
df_links.shape

(998750, 6)

In [53]:
df_links.product1.nunique()

143483

In [54]:
df_links.product2.nunique()

143483

#### ---- Group all the views of each product in lists

In [55]:
# for each product create a list with the 'views'
df_links_groups = df_links.groupby(['product1', 'subcategory_p1', 'category_p1'], as_index=False)['product2', 'subcategory_p2', 'category_p2'].agg(lambda x: x.tolist())

In [56]:
df_links_groups.shape

(124366, 6)

In [57]:
# create a column with the number of 'views' for each product
df_links_groups['N_products'] = df_links_groups['product2'].apply(lambda x: len(x))

In [58]:
df_links_groups.head(2)

,product1,subcategory_p1,category_p1,product2,subcategory_p2,category_p2,N_products
0,3,1171.0,938.0,"[336863, 86962]","[1171.0, 1171.0]","[938.0, 938.0]",2
1,6,1091.0,573.0,"[318406, 118177, 384758, 6, 236992, 397335, 413179, 182745, 174778, 453043, 446534, 411580, 25123]","[1091.0, 1091.0, 1091.0, 1091.0, 535.0, 1091.0, 1666.0, 1091.0, 406.0, 1091.0, 1090.0, 1091.0, 1091.0]","[573.0, 573.0, 573.0, 573.0, 426.0, 573.0, 1684.0, 573.0, 872.0, 573.0, 648.0, 573.0, 573.0]",13


In [59]:
df_links_groups.describe()

,product1,subcategory_p1,category_p1,N_products
count,124366.000000,124366.000000,124366.000000,124366.000000
mean,233290.915250,887.814419,847.997692,7.582571
std,134638.373328,480.619926,499.240926,13.901208
min,3.000000,0.000000,8.000000,1.000000
25%,116262.500000,478.000000,440.000000,1.000000
50%,233513.500000,936.000000,854.000000,3.000000
75%,349797.500000,1277.000000,1308.000000,8.000000
max,466864.000000,1697.000000,1698.000000,743.000000


In [60]:
df_links_groups.to_pickle('data/final/df_links_window_size_'+str(window_size)+'new.pkl')

In [61]:
df_links_groups = pd.read_pickle('data/final/df_links_window_size_'+str(window_size)+'new.pkl')

## -- Product details

In [62]:
df_item_properties.head()

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566
3,1431226800000,59481,790,n15360.000
4,1431831600000,156781,917,828513


In [63]:
df_item_properties.shape

(20275902, 4)

In [64]:
df_item_properties.itemid.nunique()

417053

### --- Keep only 6 main categories

I will keep products of 6 main categories. This 6 main categories will have around 30.000 products splitted in many subcategories. These 6 categories should also be in the 'df_links_groups' dataset that I created above.

- First find categories for which I have many products in the 'df_links_groups' dataset. And these products have many 'links'

In [65]:
df_links_groups.head(2)

,product1,subcategory_p1,category_p1,product2,subcategory_p2,category_p2,N_products
0,3,1171.0,938.0,"[336863, 86962]","[1171.0, 1171.0]","[938.0, 938.0]",2
1,6,1091.0,573.0,"[318406, 118177, 384758, 6, 236992, 397335, 413179, 182745, 174778, 453043, 446534, 411580, 25123]","[1091.0, 1091.0, 1091.0, 1091.0, 535.0, 1091.0, 1666.0, 1091.0, 406.0, 1091.0, 1090.0, 1091.0, 1091.0]","[573.0, 573.0, 573.0, 573.0, 426.0, 573.0, 1684.0, 573.0, 872.0, 573.0, 648.0, 573.0, 573.0]",13


In [66]:
# 594.,  561.,  986., 1531., 1181., 1426.

In [67]:
# count the number of products for which we have views and the average number of views
df_links_categs1 = df_links_groups.groupby('category_p1', as_index=False).agg({'product1':'count', 'N_products':'mean'})
df_links_categs1 = df_links_categs1.sort_values(by='product1', ascending=False)
df_links_categs1.head()

,category_p1,product1,N_products
97,587.0,5033,3.110471
84,500.0,4112,7.633998
46,250.0,3518,6.978113
90,561.0,2967,10.496124
265,1606.0,2615,16.487954


In [68]:
# count the number of all products in each main category
df_categs_groups = df_categs.groupby('parentid', as_index=False).itemid.count()
#df_top6_categs = df_categs_groups.sort_values(by='itemid', ascending=False)
df_categs_groups.head()

,parentid,itemid
0,8,1115
1,9,284
2,14,1793
3,19,3
4,20,1803


In [69]:
# merge the above dfs
df_links_categs1 = df_links_categs1.merge(df_categs_groups, left_on='category_p1', right_on='parentid', how='left')
df_links_categs1 = df_links_categs1.drop(['parentid'], axis=1)
df_links_categs1.columns = ['category', 'N_products', 'Avg_links', 'All_products']
df_links_categs1.head()

,category,N_products,Avg_links,All_products
0,587.0,5033,3.110471,21128
1,500.0,4112,7.633998,20887
2,250.0,3518,6.978113,9163
3,561.0,2967,10.496124,5631
4,1606.0,2615,16.487954,6374


In [70]:
# 594.,  561.,  986., 1531., 1181., 1426.

In [71]:
category_ids = [955.0, 1606.0, 561.0, 105.0, 1426.0, 1308.0]

In [72]:
df_links_categs1[df_links_categs1.category.isin(category_ids)]

,category,N_products,Avg_links,All_products
3,561.0,2967,10.496124,5631
4,1606.0,2615,16.487954,6374
5,105.0,2511,6.968538,4573
7,1426.0,2470,9.769636,4678
8,955.0,2443,17.102743,4131
12,1308.0,1849,12.497025,2854


In [73]:
df_links_categs1[df_links_categs1.category.isin(category_ids)].All_products.sum()

28241

In [74]:
# create a list with the item ids that belong in the 6 selected categories
item_ids = df_categs[df_categs.parentid.isin(category_ids)].itemid.tolist()
len(item_ids)

28241

### --- Keep sessions for products that belong to the 6 selected categories

In [75]:
# keep only products of the selected main categories
df_links_groups2 = df_links_groups[df_links_groups.category_p1.isin(category_ids)]

In [76]:
df_links_groups2.product1.nunique()

14855

In [77]:
# count number of products per main category
df_links_groups3 = df_links_groups2.groupby('category_p1', as_index=False).product1.count()
df_links_groups3 = df_links_groups3.sort_values(by='product1', ascending=False).head(20)
df_links_groups3

,category_p1,product1
1,561.0,2967
5,1606.0,2615
0,105.0,2511
4,1426.0,2470
2,955.0,2443
3,1308.0,1849


In [78]:
# save the sessions of the products that belong to the 6 selected categories
df_links_groups2.to_pickle('data/final/sessions/df_sessions_retailrocket_window_size_'+str(window_size)+'new.pkl')

### --- Keep details only of the products that belong in the 6 selected categories

In [106]:
df_item_properties.head(2)

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200


In [107]:
# keep only details of products that belong to the 6 categories
df_selected_items = df_item_properties[df_item_properties.itemid.isin(item_ids)]

In [108]:
# keep only one row for each pair (itemid, property)
df_selected_items = df_selected_items.drop_duplicates(subset=['itemid', 'property'], keep='first')

In [109]:
# create a df with the category property
df_item_categ = df_selected_items[df_selected_items.property=='categoryid'][['itemid', 'value']]
df_item_categ.columns = ['itemid', 'categoryid']

# create a df with the title property
df_item_titles = df_selected_items[df_selected_items.property=='283'][['itemid', 'value']]
df_item_titles.columns = ['itemid', 'title']

# merge the above dataframes
df_item_details = df_item_titles.merge(df_item_categ, left_on='itemid', right_on='itemid', how='left')

In [110]:
df_item_details.head(2)

,itemid,title,categoryid
0,291285,942411 177151 64816 133116 1128577 227976 1196328 603188 1128577 227976 1196328 177151 64816 737007 992688 727274 1273256 574225 567967 269926 348137 786181 709553 901561 1080764 824998 111677 675499 113294 610517 398124 280445,1589
1,103012,1249027 820477 72034 912996 1128577 404632 30603 1009622 1128577 780582 904144 716033 820477 375545 n72000.000 590873 611917 606886 1128577 954367 631756 7681 353870 1324984 145012 568015 737771 237874 1249027 1271914 1058996 836265,1483


In [111]:
# convert column to int
df_item_details['categoryid'] = df_item_details['categoryid'].astype('int')
df_cat_tree['categoryid'] = df_cat_tree['categoryid'].astype('int')
df_cat_tree = df_cat_tree.dropna(subset=['parentid'])
df_cat_tree['parentid'] = df_cat_tree['parentid'].astype('int')
# parentid


# add the main category
df_item_details = df_item_details.merge(df_cat_tree, left_on='categoryid', right_on='categoryid', how='left')

C:\Users\Dimos\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [112]:
df_item_details.head(2)

,itemid,title,categoryid,parentid
0,291285,942411 177151 64816 133116 1128577 227976 1196328 603188 1128577 227976 1196328 177151 64816 737007 992688 727274 1273256 574225 567967 269926 348137 786181 709553 901561 1080764 824998 111677 675499 113294 610517 398124 280445,1589,1426
1,103012,1249027 820477 72034 912996 1128577 404632 30603 1009622 1128577 780582 904144 716033 820477 375545 n72000.000 590873 611917 606886 1128577 954367 631756 7681 353870 1324984 145012 568015 737771 237874 1249027 1271914 1058996 836265,1483,561


In [113]:
# rename columns
df_item_details.columns = ['itemid', 'Title', 'SubCategory', 'Category']

In [114]:
# convert columns to str
df_item_details['SubCategory'] = df_item_details['SubCategory'].astype('int')
df_item_details['Category'] = df_item_details['Category'].astype('int')

In [115]:
# create a column with Category+'_cat'
df_item_details['Category2'] = df_item_details['Category'].apply(lambda x: str(x)+'_cat')

# create a column with SubCategory+'_subcat'
df_item_details['SubCategory2'] = df_item_details['SubCategory'].apply(lambda x: str(x)+'_subcat')

In [116]:
# create a column with the text: Title+Category2+SubCategory2
df_item_details['Title_lc2'] = df_item_details.apply(lambda x: x['Title']+' '+x['Category2']+' '+x['SubCategory2'], axis=1)

In [117]:
df_item_details.head(2)

,itemid,Title,SubCategory,Category,Category2,SubCategory2,Title_lc2
0,291285,942411 177151 64816 133116 1128577 227976 1196328 603188 1128577 227976 1196328 177151 64816 737007 992688 727274 1273256 574225 567967 269926 348137 786181 709553 901561 1080764 824998 111677 675499 113294 610517 398124 280445,1589,1426,1426_cat,1589_subcat,942411 177151 64816 133116 1128577 227976 1196328 603188 1128577 227976 1196328 177151 64816 737007 992688 727274 1273256 574225 567967 269926 348137 786181 709553 901561 1080764 824998 111677 675499 113294 610517 398124 280445 1426_cat 1589_subcat
1,103012,1249027 820477 72034 912996 1128577 404632 30603 1009622 1128577 780582 904144 716033 820477 375545 n72000.000 590873 611917 606886 1128577 954367 631756 7681 353870 1324984 145012 568015 737771 237874 1249027 1271914 1058996 836265,1483,561,561_cat,1483_subcat,1249027 820477 72034 912996 1128577 404632 30603 1009622 1128577 780582 904144 716033 820477 375545 n72000.000 590873 611917 606886 1128577 954367 631756 7681 353870 1324984 145012 568015 737771 237874 1249027 1271914 1058996 836265 561_cat 1483_subcat


In [118]:
df_item_details.shape

(28241, 7)

In [119]:
df_item_details.Category2.nunique()

6

In [120]:
df_item_details.SubCategory.nunique()

37

In [121]:
df_item_details.to_pickle('data/final/product_details/df_product_details_retailrocket.pkl')

In [3]:
df_item_details = pd.read_pickle('data/final/product_details/df_product_details_retailrocket.pkl')

In [4]:
df_item_details.head(1)

,itemid,Title,SubCategory,Category,Category2,SubCategory2,Title_lc2
0,291285,942411 177151 64816 133116 1128577 227976 1196328 603188 1128577 227976 1196328 177151 64816 737007 992688 727274 1273256 574225 567967 269926 348137 786181 709553 901561 1080764 824998 111677 675499 113294 610517 398124 280445,1589,1426,1426_cat,1589_subcat,942411 177151 64816 133116 1128577 227976 1196328 603188 1128577 227976 1196328 177151 64816 737007 992688 727274 1273256 574225 567967 269926 348137 786181 709553 901561 1080764 824998 111677 675499 113294 610517 398124 280445 1426_cat 1589_subcat


In [6]:
df_item_details.SubCategory.nunique()

37